In [569]:
import pandas as pd
import numpy as np
import re
from tabulate import tabulate
import time 
import datetime

import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
from sklearn import *

import sklearn.externals
import joblib

from imblearn.over_sampling import SMOTE


import sklearn.neighbors._base
import warnings
warnings.filterwarnings('ignore')

% matplotlib inline

UsageError: Line magic function `%` not found.


In [570]:
## Reading the csv file
reviews = pd.read_csv("sample30.csv")

In [571]:
## Displaying the head
reviews.head()

,id,brand,categories,manufacturer,name,reviews_date,reviews_didPurchase,reviews_doRecommend,reviews_rating,reviews_text,reviews_title,reviews_userCity,reviews_userProvince,reviews_username,user_sentiment
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",Universal Music Group / Cash Money,Pink Friday: Roman Reloaded Re-Up (w/dvd),2012-11-30T06:21:45.000Z,NaN,NaN,5,i love this album. it's very good. more to the...,Just Awesome,Los Angeles,NaN,joshua,Positive
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor. This review was collected as part...,Good,NaN,NaN,dorothy w,Positive
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor.,Good,NaN,NaN,dorothy w,Positive
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-01-06T00:00:00.000Z,False,False,1,I read through the reviews on here before look...,Disappointed,NaN,NaN,rebecca,Negative
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-12-21T00:00:00.000Z,False,False,1,My husband bought this gel for us. The gel cau...,Irritation,NaN,NaN,walker557,Negative


# Data Cleansing and Pre-processing

In [572]:
## Checking the shape of the data frame
reviews.shape

(30000, 15)

In [573]:
## Checking the info of the data set
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 15 columns):
id                      30000 non-null object
brand                   30000 non-null object
categories              30000 non-null object
manufacturer            29859 non-null object
name                    30000 non-null object
reviews_date            29954 non-null object
reviews_didPurchase     15932 non-null object
reviews_doRecommend     27430 non-null object
reviews_rating          30000 non-null int64
reviews_text            30000 non-null object
reviews_title           29810 non-null object
reviews_userCity        1929 non-null object
reviews_userProvince    170 non-null object
reviews_username        29937 non-null object
user_sentiment          29999 non-null object
dtypes: int64(1), object(14)
memory usage: 3.4+ MB


In [574]:
## Checking for the null value counts
reviews.isnull().sum()

id                          0
brand                       0
categories                  0
manufacturer              141
name                        0
reviews_date               46
reviews_didPurchase     14068
reviews_doRecommend      2570
reviews_rating              0
reviews_text                0
reviews_title             190
reviews_userCity        28071
reviews_userProvince    29830
reviews_username           63
user_sentiment              1
dtype: int64

In [575]:
## Checking the Null value count percentages
round(100*reviews.isnull().sum()/reviews.shape[0],2)

id                       0.00
brand                    0.00
categories               0.00
manufacturer             0.47
name                     0.00
reviews_date             0.15
reviews_didPurchase     46.89
reviews_doRecommend      8.57
reviews_rating           0.00
reviews_text             0.00
reviews_title            0.63
reviews_userCity        93.57
reviews_userProvince    99.43
reviews_username         0.21
user_sentiment           0.00
dtype: float64

In [576]:
## Dropping the columns reviews_userCity and reviews_userProvince as they have more than 90% null values and seems to be no importance for the analysis
reviews = reviews.drop(['reviews_userCity','reviews_userProvince'],axis=1)
reviews.shape

(30000, 13)

In [577]:
## checking again the null value percentages
round(100*reviews.isnull().sum()/reviews.shape[0],2)

id                      0.00
brand                   0.00
categories              0.00
manufacturer            0.47
name                    0.00
reviews_date            0.15
reviews_didPurchase    46.89
reviews_doRecommend     8.57
reviews_rating          0.00
reviews_text            0.00
reviews_title           0.63
reviews_username        0.21
user_sentiment          0.00
dtype: float64

In [578]:
## Displaying the head
reviews.head()

,id,brand,categories,manufacturer,name,reviews_date,reviews_didPurchase,reviews_doRecommend,reviews_rating,reviews_text,reviews_title,reviews_username,user_sentiment
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",Universal Music Group / Cash Money,Pink Friday: Roman Reloaded Re-Up (w/dvd),2012-11-30T06:21:45.000Z,NaN,NaN,5,i love this album. it's very good. more to the...,Just Awesome,joshua,Positive
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor. This review was collected as part...,Good,dorothy w,Positive
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor.,Good,dorothy w,Positive
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-01-06T00:00:00.000Z,False,False,1,I read through the reviews on here before look...,Disappointed,rebecca,Negative
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-12-21T00:00:00.000Z,False,False,1,My husband bought this gel for us. The gel cau...,Irritation,walker557,Negative


In [579]:
## Checking the user_sentiment which has null value
reviews[reviews['user_sentiment'].isnull()]

,id,brand,categories,manufacturer,name,reviews_date,reviews_didPurchase,reviews_doRecommend,reviews_rating,reviews_text,reviews_title,reviews_username,user_sentiment
28354,AVpfRTh1ilAPnD_xYic2,Disney,"Movies, Music & Books,Movies,Kids' & Family,Wa...",Walt Disney,Planes: Fire Rescue (2 Discs) (includes Digita...,2014-11-07T00:00:00.000Z,NaN,True,5,my kids absolutely loved this film so much tha...,a super hit with my children. they loved it!!??,7.87E+11,NaN


## Imputing default values for the Null Values

In [580]:
## There are Null values in manufacturer column and we can impute the Brand values as Brand and Manufacturer seems to have similarity
reviews['manufacturer'] = np.where(reviews['manufacturer'].isnull(),reviews['brand'],reviews['manufacturer'])

## There is one record with user_sengtiment as Null and imputed manually with Positive based on the reviews_rating column.
reviews['user_sentiment'] = np.where(reviews['user_sentiment'].isnull(),'Positive',reviews['user_sentiment'])

## reviews_didPurchase and reviews_doRecommend has Null values and I will impute them with "N/A" 
reviews['reviews_didPurchase'] = reviews['reviews_didPurchase'].fillna('N/A')
reviews['reviews_doRecommend'] = reviews['reviews_doRecommend'].fillna('N/A')

## We could see there is null values for Reviews_date and can be imputed with default date.
reviews['reviews_date'] = reviews['reviews_date'].fillna('1969-12-31T00:00:00.000Z')

## There are some records with reviews_username as Null and i will impute with "Anonymus" user.
reviews['reviews_username'] = reviews['reviews_username'].fillna('Anonymus')

## Imputing Niull values in reviews_title with reviews_text as title also has similar text
reviews['reviews_title'] = np.where(reviews['reviews_title'].isnull(),reviews['reviews_text'],reviews['reviews_title'])



In [581]:
def is_date_matching(date_str):
    try:
        return datetime.datetime.strptime(date_str, '%Y-%m-%d')
    except ValueError:
        return datetime.datetime.strptime('1969-12-31', '%Y-%m-%d')

In [582]:
## Extracting date part alone as timestamp part is not required for our analysis
reviews['reviews_date'] = reviews['reviews_date'].apply(lambda x: x.split('T')[0])
## Cleaning date column with proper default date and creating reviews_year column for analysis
reviews['reviews_date'] = reviews['reviews_date'].apply(lambda x: is_date_matching(x))
reviews['reviews_year'] = reviews['reviews_date'].apply(lambda x: x.year)

In [583]:
## Checking for the transformed columns
reviews.head()

,id,brand,categories,manufacturer,name,reviews_date,reviews_didPurchase,reviews_doRecommend,reviews_rating,reviews_text,reviews_title,reviews_username,user_sentiment,reviews_year
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",Universal Music Group / Cash Money,Pink Friday: Roman Reloaded Re-Up (w/dvd),2012-11-30,N/A,N/A,5,i love this album. it's very good. more to the...,Just Awesome,joshua,Positive,2012
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09,True,N/A,5,Good flavor. This review was collected as part...,Good,dorothy w,Positive,2017
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09,True,N/A,5,Good flavor.,Good,dorothy w,Positive,2017
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-01-06,False,False,1,I read through the reviews on here before look...,Disappointed,rebecca,Negative,2016
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-12-21,False,False,1,My husband bought this gel for us. The gel cau...,Irritation,walker557,Negative,2016


## Tranforming Target column

In [584]:
## Tranforming the user_sentiment from Categorical to Numeric as this is our prediction variable
reviews['user_sentiment'] = reviews['user_sentiment'].map({'Positive': 1, 'Negative': 0}).astype('int')

In [585]:
## Lets check again info of the data
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 14 columns):
id                     30000 non-null object
brand                  30000 non-null object
categories             30000 non-null object
manufacturer           30000 non-null object
name                   30000 non-null object
reviews_date           30000 non-null datetime64[ns]
reviews_didPurchase    30000 non-null object
reviews_doRecommend    30000 non-null object
reviews_rating         30000 non-null int64
reviews_text           30000 non-null object
reviews_title          30000 non-null object
reviews_username       30000 non-null object
user_sentiment         30000 non-null int64
reviews_year           30000 non-null int64
dtypes: datetime64[ns](1), int64(3), object(10)
memory usage: 3.2+ MB


# Model Building

In [663]:
## Assigning X,y,seed values for the model building 
seed = 42
X = reviews['reviews_text']
y = reviews['user_sentiment']

In [664]:
## Pre-processing the reviews_text
## Keeping only alphabets which are necessary for classification. Also convert the text to lower case
X = X.apply(lambda x: re.sub('[^a-zA-Z]+', ' ', x.lower()).strip())

In [668]:
nltk.download('wordnet')

[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1076)>


False

In [666]:
## Applying Lemmatization to the review_text
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# tokenise text
wordnet_lemmatizer = WordNetLemmatizer()
X = X.apply(lambda x: ' '.join([wordnet_lemmatizer.lemmatize(token, pos='n') for token in word_tokenize(x)]))
#X = X.apply(lambda x: ' '.join([Word(word).lemmatize() for word in x.split()]))

LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - '/Users/vlatha/nltk_data'
    - '/opt/anaconda3/nltk_data'
    - '/opt/anaconda3/share/nltk_data'
    - '/opt/anaconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


# Feature Extraction


In [590]:
## Getting the stowords from nltk for Text analysis
stop_words = set(stopwords.words('english'))

## TF-IDF

In [591]:
## Getting the features
tfidf_vec = TfidfVectorizer(
    max_features=None, 
    lowercase=True, 
    analyzer='word', 
    stop_words= stop_words, 
    ngram_range=(1, 1))


# Logistic Regression

## Lets now check if we have class imbalance

In [592]:
# Checking class imbalnce is present or not
y.value_counts()

1    26633
0     3367
Name: user_sentiment, dtype: int64

## Above clearly shows there is imbalance in the sentiments. I will apply SMOTE technique to overcome that after splitting into Train and Test daat sets.

In [593]:
## Splitting the data to train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=seed)

## Applying TF-IDF to extract train, test feature set
tfidf_vec.fit(X_train)
X_train = tfidf_vec.transform(X_train)
X_test = tfidf_vec.transform(X_test)

## Applying SMOTE for imbalance data
sm = SMOTE()
X_train_smote, y_train_smote = sm.fit_sample(X_train, y_train)
X_test_smote, y_test_smote = sm.fit_sample(X_test, y_test)


In [594]:
## Logistic Regression
model = LogisticRegression(penalty="l2", random_state=seed, max_iter=500)

## Model fit
model.fit(X_train_smote, y_train_smote)

## Model Prediction
y_pred = model.predict(X_test_smote)

In [595]:
## Accuracy
train_accuracy = model.score(X_train_smote, y_train_smote)
test_accuracy = model.score(X_test_smote, y_test_smote)

## Classification report
model_performance = classification_report(y_test_smote, y_pred)

y_pred_proba = model.predict_proba(X_test_smote)
roc_auc = roc_auc_score(y_test_smote, y_pred_proba[:,1])


print('')
print(model_performance)
print('')

print ('Training Accuracy: ', round(train_accuracy,2))
print ('Testing Accuracy: ', round(test_accuracy,2))
print('ROC_AUC score: ', round(roc_auc,2))


              precision    recall  f1-score   support

           0       0.93      0.93      0.93      5349
           1       0.93      0.93      0.93      5349

    accuracy                           0.93     10698
   macro avg       0.93      0.93      0.93     10698
weighted avg       0.93      0.93      0.93     10698


Training Accuracy:  0.96
Testing Accuracy:  0.93
ROC_AUC score:  0.97


## Save the ML Model

In [596]:
# Save the model as a pickle in a file 
joblib.dump(model, 'models/lr_model.pkl') 
# Save the TF-IDF model as a pickle in a file 
joblib.dump(tfidf_vec, 'models/tfidf_model.pkl') 

['models/tfidf_model.pkl']

## Load the ML Model

In [597]:
# Load the model from the file 
lr_pickle_model = joblib.load('models/lr_model.pkl') 
# Load the model from the file 
tfidf_pickle_model = joblib.load('models/tfidf_model.pkl') 

In [598]:
# Use the loaded model to make predictions
response_lr_model = lr_pickle_model.predict(X_test)

## Evaluate the Loaded Model

In [599]:
# predict sentiment on test data
y_pred_load = lr_pickle_model.predict(X_test)

# create onfusion matrix
cm_load = confusion_matrix(y_test, y_pred_load)
print(cm_load)

#Checking the accuracy
print("Accuracy: \t",round(lr_pickle_model.score(X_test, y_test),2))

# check area under curve
y_pred_prob_load = lr_pickle_model.predict_proba(X_test)[:, 1]
print("ROC:    \t", round(roc_auc_score(y_test, y_pred_prob_load),2))

[[ 579   72]
 [ 388 4961]]
Accuracy: 	 0.92
ROC:    	 0.97


# Product Recommendation

### Now lets get the top 15 products from the Recommendation system.
### We are going to execute Item-based Recommendation system



### Dividing the dataset into train and test

In [600]:
## Taking the required columns - username,brand(product) and ratings from the data set
df_recommend = pd.DataFrame(data = reviews[['reviews_username','id','reviews_rating']])
## Checking the data set
df_recommend.head()

,reviews_username,id,reviews_rating
0,joshua,AV13O1A8GV-KLJ3akUyj,5
1,dorothy w,AV14LG0R-jtxr-f38QfS,5
2,dorothy w,AV14LG0R-jtxr-f38QfS,5
3,rebecca,AV16khLE-jtxr-f38VFn,1
4,walker557,AV16khLE-jtxr-f38VFn,1


In [601]:
## If there are any duplicate ratings present for user and product combination then lets apply mean and take the average rating
df_recommend = df_recommend.groupby(['reviews_username','id'],as_index=False).agg({'reviews_rating':pd.Series.mode,'reviews_rating':np.mean})

In [602]:
## Changing column names for better understanding and representation
df_recommend['username'] = df_recommend['reviews_username']
df_recommend['product'] = df_recommend['id']
df_recommend['rating'] = round(df_recommend['reviews_rating'],1)
## Dropping the old columns and checking the data
df_recommend = df_recommend.drop(['reviews_username','id','reviews_rating'],axis=1)
df_recommend.head()

,username,product,rating
0,00dog3,AVpe6FfKilAPnD_xQmHi,4.0
1,00sab00,AVpe41TqilAPnD_xQH3d,1.0
2,00sab00,AVpe59io1cnluZ0-ZgDU,3.0
3,01impala,AVpfPaoqLJeJML435Xk9,3.0
4,02dakota,AVpfPaoqLJeJML435Xk9,5.0


In [603]:
## Splitting the data into train and test sets
train, test = train_test_split(df_recommend, test_size=0.2, random_state=seed)
## check the shape of the data sets
print(train.shape)
print(test.shape)

(22084, 3)
(5521, 3)


In [604]:
## Checking info of the data frame
df_recommend.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27605 entries, 0 to 27604
Data columns (total 3 columns):
username    27605 non-null object
product     27605 non-null object
rating      27605 non-null float64
dtypes: float64(1), object(2)
memory usage: 862.7+ KB


In [605]:
# Apply pivot to see the user ratings for each product and give '0' wherever rating is not provided by the user
df_product_features = train.pivot(
    index='username',
    columns='product',
    values='rating'
).fillna(0)

In [606]:
## Checking the data set after pivoting the data frame
df_product_features.head()

product,AV13O1A8GV-KLJ3akUyj,AV14LG0R-jtxr-f38QfS,AV16khLE-jtxr-f38VFn,AV1YGDqsGV-KLJ3adc-O,AV1YlENIglJLPUi8IHsX,AV1YmBrdGV-KLJ3adewb,AV1YmDL9vKc47QAVgr7_,AV1Ymf_rglJLPUi8II2v,AV1Yn94nvKc47QAVgtst,AV1YnUMYglJLPUi8IJpK,...,AVpfrfHF1cnluZ0-pRai,AVpfrgjFLJeJML43BvCc,AVpfs0tUilAPnD_xgqN2,AVpfsQoeilAPnD_xgfx5,AVpfshNsLJeJML43CB8q,AVpfthSailAPnD_xg3ON,AVpftikC1cnluZ0-p31V,AVpftymALJeJML43CZ6y,AVpfv4TlilAPnD_xhjNS,AVpfvieo1cnluZ0-qdnu
username,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
02dakota,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
02deuce,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
06stidriver,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Copy train and test dataset

### These dataset will be used for prediction and evaluation.


*   Dummy train will be used later for prediction of the products which has not been rated by the user. To ignore the products rated by the user, we will mark it as 0 during prediction. The products not rated by user is marked as 1 for prediction.
*   Dummy test will be used for evaluation. To evaluate, we will only make prediction on the products rated by the user. So, this is marked as 1. This is just opposite of dummy_train



In [607]:
## Taking the copy of train and test data sets
dummy_train = train.copy()
dummy_test = test.copy()

In [608]:
## Changing the rating value to '0' if the user provided rating
dummy_train['rating'] = dummy_train['rating'].apply(lambda x: 0 if x>=1 else 1)
## Changing the rating value to '1' if the user provided rating
dummy_test['rating'] = dummy_test['rating'].apply(lambda x: 1 if x>=1 else 0)

In [609]:
# The products not rated by user is marked as 1 for prediction. 
dummy_train = dummy_train.pivot(
    index='username',
    columns='product',
    values='rating'
).fillna(1)

# The products not rated by user is marked as 0 for evaluation. 
dummy_test = dummy_test.pivot(
    index='username',
    columns='product',
    values='rating'
).fillna(0)

In [610]:
## Lets check the train data set
dummy_train.head()

product,AV13O1A8GV-KLJ3akUyj,AV14LG0R-jtxr-f38QfS,AV16khLE-jtxr-f38VFn,AV1YGDqsGV-KLJ3adc-O,AV1YlENIglJLPUi8IHsX,AV1YmBrdGV-KLJ3adewb,AV1YmDL9vKc47QAVgr7_,AV1Ymf_rglJLPUi8II2v,AV1Yn94nvKc47QAVgtst,AV1YnUMYglJLPUi8IJpK,...,AVpfrfHF1cnluZ0-pRai,AVpfrgjFLJeJML43BvCc,AVpfs0tUilAPnD_xgqN2,AVpfsQoeilAPnD_xgfx5,AVpfshNsLJeJML43CB8q,AVpfthSailAPnD_xg3ON,AVpftikC1cnluZ0-p31V,AVpftymALJeJML43CZ6y,AVpfv4TlilAPnD_xhjNS,AVpfvieo1cnluZ0-qdnu
username,,,,,,,,,,,,,,,,,,,,,
00dog3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
00sab00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
02dakota,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
02deuce,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
06stidriver,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [611]:
## Lets check the test data set
dummy_test.head()

product,AV16khLE-jtxr-f38VFn,AV1YGDqsGV-KLJ3adc-O,AV1YIch7GV-KLJ3addeG,AV1YlENIglJLPUi8IHsX,AV1YmBrdGV-KLJ3adewb,AV1YmDL9vKc47QAVgr7_,AV1Ymf_rglJLPUi8II2v,AV1Yn94nvKc47QAVgtst,AV1Ynb3bglJLPUi8IJxJ,AV1YneDPglJLPUi8IJyQ,...,AVpfpM2yilAPnD_xfmDG,AVpfqW4WilAPnD_xf7a_,AVpfr5cb1cnluZ0-pZFp,AVpfrFDZLJeJML43Bmv0,AVpfrTyiLJeJML43BrSI,AVpfrfHF1cnluZ0-pRai,AVpfs0tUilAPnD_xgqN2,AVpfthSailAPnD_xg3ON,AVpftikC1cnluZ0-p31V,AVpfvieo1cnluZ0-qdnu
username,,,,,,,,,,,,,,,,,,,,,
01impala,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0325home,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123cat123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123too,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Item Based Similarity

In [612]:
## Taking the transpose of the reviews matrix to normalize the rating around the mean for different product ID. 
## In the user based similarity, we had taken mean for each user intead of each product. 
product_features = train.pivot(
    index='username',
    columns='product',
    values='rating'
).T

product_features.head()

username,00dog3,00sab00,02dakota,02deuce,06stidriver,08dallas,09mommy11,1.11E+24,1085,10ten,...,zpalma,zsarah,zsazsa,zt313,zubb,zuttle,zwithanx,zxjki,zzdiane,zzz1127
product,,,,,,,,,,,,,,,,,,,,,
AV13O1A8GV-KLJ3akUyj,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AV14LG0R-jtxr-f38QfS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AV16khLE-jtxr-f38VFn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AV1YGDqsGV-KLJ3adc-O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AV1YlENIglJLPUi8IHsX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [613]:
## Normalising the product rating for each product
mean = np.nanmean(product_features, axis=1)
df_subtracted = (product_features.T-mean).T

In [614]:
df_subtracted.head()

username,00dog3,00sab00,02dakota,02deuce,06stidriver,08dallas,09mommy11,1.11E+24,1085,10ten,...,zpalma,zsarah,zsazsa,zt313,zubb,zuttle,zwithanx,zxjki,zzdiane,zzz1127
product,,,,,,,,,,,,,,,,,,,,,
AV13O1A8GV-KLJ3akUyj,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AV14LG0R-jtxr-f38QfS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AV16khLE-jtxr-f38VFn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AV1YGDqsGV-KLJ3adc-O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AV1YlENIglJLPUi8IHsX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [615]:
## Finding the cosine similarity. Note that since the data is normalised, both the cosine metric and correlation metric will give the same value. 
from sklearn.metrics.pairwise import pairwise_distances

# User Similarity Matrix
item_correlation = 1 - pairwise_distances(df_subtracted.fillna(0), metric='cosine')
item_correlation[np.isnan(item_correlation)] = 0
print(item_correlation)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [616]:
## Filtering the correlation only for which the value is greater than 0. (Positively correlated)
item_correlation[item_correlation<0]=0
item_correlation

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

# Prediction

In [617]:
item_predicted_ratings = np.dot((product_features.fillna(0).T),item_correlation)
item_predicted_ratings

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.03458652,
        0.        ],
       [0.        , 0.        , 0.01153019, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.0029185 ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.0576442 ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.00583699]])

In [618]:
item_predicted_ratings.shape

(20254, 260)

In [619]:
dummy_train.shape

(20254, 260)

### Filtering the rating only for the products not rated by the user for recommendation

In [620]:
item_final_rating = np.multiply(item_predicted_ratings,dummy_train)
item_final_rating.head()

product,AV13O1A8GV-KLJ3akUyj,AV14LG0R-jtxr-f38QfS,AV16khLE-jtxr-f38VFn,AV1YGDqsGV-KLJ3adc-O,AV1YlENIglJLPUi8IHsX,AV1YmBrdGV-KLJ3adewb,AV1YmDL9vKc47QAVgr7_,AV1Ymf_rglJLPUi8II2v,AV1Yn94nvKc47QAVgtst,AV1YnUMYglJLPUi8IJpK,...,AVpfrfHF1cnluZ0-pRai,AVpfrgjFLJeJML43BvCc,AVpfs0tUilAPnD_xgqN2,AVpfsQoeilAPnD_xgfx5,AVpfshNsLJeJML43CB8q,AVpfthSailAPnD_xg3ON,AVpftikC1cnluZ0-p31V,AVpftymALJeJML43CZ6y,AVpfv4TlilAPnD_xhjNS,AVpfvieo1cnluZ0-qdnu
username,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000
00sab00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.005956,0.0,...,0.008078,0.0,0.0,0.0,0.0,0.038316,0.002263,0.0,0.034587,0.000000
02dakota,0.0,0.0,0.011530,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000
02deuce,0.0,0.0,0.009224,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000
06stidriver,0.0,0.0,0.000000,0.003866,0.0,0.0,0.0,0.002081,0.003646,0.0,...,0.002472,0.0,0.0,0.0,0.0,0.011727,0.001303,0.0,0.000000,0.007296


## Top 15 prediction for the user -1

In [621]:
item_final_rating.iloc[1].sort_values(ascending=False)[0:15]

product
AVpfE7puilAPnD_xUcCW    0.120537
AVpfNc9cLJeJML434tza    0.042088
AVpf5ujMilAPnD_xkQVi    0.040216
AVpfoSS51cnluZ0-oVH9    0.039377
AVpfthSailAPnD_xg3ON    0.038316
AVpfv4TlilAPnD_xhjNS    0.034587
AVpe6FfKilAPnD_xQmHi    0.031271
AVpe6PCDLJeJML43yFQH    0.029658
AVpfOmKwLJeJML435GM7    0.025282
AVpfJcHuLJeJML433ZrY    0.023106
AVpf0thK1cnluZ0-r8vR    0.021604
AVpf7aUAilAPnD_xkqbP    0.018944
AVpf9_3MLJeJML43GvdB    0.018559
AVpe8gsILJeJML43y6Ed    0.017407
AVpe31o71cnluZ0-YrSD    0.016873
Name: 00sab00, dtype: float64

# Evaluation

### Here lets evaluate for the product which is already rated by the user insead of predicting it for the product not rated by the user. 

# Using Item similarity

In [622]:
test_product_features = test.pivot(
    index='username',
    columns='product',
    values='rating'
).T

mean = np.nanmean(test_product_features, axis=1)
test_df_subtracted = (test_product_features.T-mean).T

test_item_correlation = 1 - pairwise_distances(test_df_subtracted.fillna(0), metric='cosine')
test_item_correlation[np.isnan(test_item_correlation)] = 0
test_item_correlation[test_item_correlation<0]=0

In [623]:
test_item_correlation.shape

(193, 193)

In [624]:
test_product_features.shape

(193, 5355)

In [625]:
test_item_predicted_ratings = (np.dot(test_item_correlation, test_product_features.fillna(0))).T
test_item_final_rating = np.multiply(test_item_predicted_ratings,dummy_test)
test_item_final_rating.head()

product,AV16khLE-jtxr-f38VFn,AV1YGDqsGV-KLJ3adc-O,AV1YIch7GV-KLJ3addeG,AV1YlENIglJLPUi8IHsX,AV1YmBrdGV-KLJ3adewb,AV1YmDL9vKc47QAVgr7_,AV1Ymf_rglJLPUi8II2v,AV1Yn94nvKc47QAVgtst,AV1Ynb3bglJLPUi8IJxJ,AV1YneDPglJLPUi8IJyQ,...,AVpfpM2yilAPnD_xfmDG,AVpfqW4WilAPnD_xf7a_,AVpfr5cb1cnluZ0-pZFp,AVpfrFDZLJeJML43Bmv0,AVpfrTyiLJeJML43BrSI,AVpfrfHF1cnluZ0-pRai,AVpfs0tUilAPnD_xgqN2,AVpfthSailAPnD_xg3ON,AVpftikC1cnluZ0-p31V,AVpfvieo1cnluZ0-qdnu
username,,,,,,,,,,,,,,,,,,,,,
01impala,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0325home,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123cat123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123too,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [626]:
test_ = test.pivot(
    index='username',
    columns='product',
    values='rating'
)

In [627]:
from sklearn.preprocessing import MinMaxScaler
from numpy import *

X  = test_item_final_rating.copy() 
X = X[X>0]

scaler = MinMaxScaler(feature_range=(1, 5))
print(scaler.fit(X))
y = (scaler.transform(X))


test_ = test.pivot(
    index='username',
    columns='product',
    values='rating'
)

# Finding total non-NaN value
total_non_nan = np.count_nonzero(~np.isnan(y))

MinMaxScaler(feature_range=(1, 5))


# Finding RMSE

In [628]:
rmse = (sum(sum((test_ - y )**2))/total_non_nan)**0.5
print(rmse)

0.38860832588224853


In [651]:
df = item_final_rating.iloc[0].sort_values(ascending=False)[:15]
lst = df.index[df.values != 0]
lst

'AVpfOmKwLJeJML435GM7'

In [629]:
def product_recommendation(username): 
    user_idx = username 
    #sorted_user_ratings
    sorted_user_predictions = pd.DataFrame(item_final_rating.loc[user_idx].sort_values(ascending=False)[:15])
    sorted_user_predictions.index.name = 'product'
    sorted_user_predictions.columns = ['user_ratings'] 
    sorted_user_predictions = sorted_user_predictions.sort_values('user_ratings', ascending=False)
    final_pred_df = sorted_user_predictions
    prod_review_df = reviews[['id','name','reviews_text']]
    user_recom = prod_review_df.join(final_pred_df,on=['id'],how="inner")
    print('\nBelow are the recommended items for user(user_id = {}):\n'.format(username))
    print(final_pred_df)
    return user_recom

In [630]:
user_recom = product_recommendation('00dog3')


Below are the recommended items for user(user_id = 00dog3):

                      user_ratings
product                           
AVpfOmKwLJeJML435GM7      0.048303
AVpe5s3RLJeJML43x4eA      0.040974
AVpe8gsILJeJML43y6Ed      0.039106
AVpe59io1cnluZ0-ZgDU      0.034743
AVpfN6Z71cnluZ0-gZmO      0.027830
AVpe41TqilAPnD_xQH3d      0.020857
AVpfAgSp1cnluZ0-b2-K      0.018936
AVpfRTh1ilAPnD_xYic2      0.012341
AVpf63aJLJeJML43F__Q      0.009283
AVpfJP1C1cnluZ0-e3Xy      0.004908
AVpf3VOfilAPnD_xjpun      0.002627
AVpf5CnILJeJML43FjaU      0.000000
AVpf4oLxLJeJML43FcxC      0.000000
AVpf4WWN1cnluZ0-s7Wj      0.000000
AVpf3okQLJeJML43FMif      0.000000


In [631]:
X_test_user = user_recom['reviews_text']
X_test_user = X_test_user.apply(lambda x: re.sub('[^a-zA-Z]+', ' ', x.lower()).strip())
X_test_user = tfidf_pickle_model.transform(X_test_user)
X_test_user.shape

(15346, 15571)

In [632]:
lr_pickle_model = joblib.load('models/lr_model.pkl') 
y_pred_user = lr_pickle_model.predict(X_test_user)

In [633]:
y_pred_user

array([0, 0, 0, ..., 1, 0, 1])

In [634]:
user_recom.shape

(15346, 4)

In [635]:
y_pred_user.shape

(15346,)

In [636]:
user_sentiment = pd.DataFrame(data=y_pred_user, columns=['user_sentiment'], 
                            index=user_recom.index.copy())

In [637]:
result = pd.merge(user_recom, user_sentiment, how ='left',left_index = True, right_index = True)


In [638]:
result.head()

,id,name,reviews_text,user_ratings,user_sentiment
1796,AVpe41TqilAPnD_xQH3d,Mike Dave Need Wedding Dates (dvd + Digital),I expected more from this movie and more from ...,0.020857,0
1797,AVpe41TqilAPnD_xQH3d,Mike Dave Need Wedding Dates (dvd + Digital),Would ABSOLUTELY NOT recommend. We could only ...,0.020857,0
1798,AVpe41TqilAPnD_xQH3d,Mike Dave Need Wedding Dates (dvd + Digital),Terrible movie with good actors. Can't believe...,0.020857,0
1799,AVpe41TqilAPnD_xQH3d,Mike Dave Need Wedding Dates (dvd + Digital),This movie is terrible with only a few funny s...,0.020857,0
1800,AVpe41TqilAPnD_xQH3d,Mike Dave Need Wedding Dates (dvd + Digital),This was a boring movie i couldnt watch 20 min...,0.020857,0


In [639]:
result.shape

(15346, 5)

In [640]:
user_sentiment.shape

(15346, 1)

In [641]:
new_df = result.groupby('name')['user_sentiment'].mean()

In [652]:
new_df.sort_values(ascending=False)[:15]

name
My Big Fat Greek Wedding 2 (blu-Ray + Dvd + Digital)                               0.938623
Pantene Pro-V Expert Collection Age Defy Conditioner                               0.933824
Avery174 Ready Index Contemporary Table Of Contents Divider, 1-8, Multi, Letter    0.904153
Clorox Disinfecting Bathroom Cleaner                                               0.887690
Planes: Fire Rescue (2 Discs) (includes Digital Copy) (blu-Ray/dvd)                0.880140
Clorox Disinfecting Wipes Value Pack Scented 150 Ct Total                          0.865652
WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black               0.833333
Bounce Dryer Sheets, Fresh Linen, 160 sheets                                       0.827586
Caress Moisturizing Body Bar Natural Silk, 4.75oz                                  0.823529
Burt's Bees Lip Shimmer, Raisin                                                    0.815578
Clear Scalp & Hair Therapy Total Care Nourishing Shampoo                   

In [643]:
reviews[reviews['reviews_username'] == '00dog3']

,id,brand,categories,manufacturer,name,reviews_date,reviews_didPurchase,reviews_doRecommend,reviews_rating,reviews_text,reviews_title,reviews_username,user_sentiment,reviews_year
3499,AVpe6FfKilAPnD_xQmHi,Chex,"Food,Packaged Foods,Snacks,Snacks, Cookies & C...","GENERAL MILLS SALES, INC.",Chex Muddy Buddies Brownie Supreme Snack Mix,2013-12-16,True,True,4,Very chocolaty taste to them. It hits the swee...,something different,00dog3,1,2013


In [644]:
result[result['id'] == 'AVpfOmKwLJeJML435GM7']

,id,name,reviews_text,user_ratings,user_sentiment
22351,AVpfOmKwLJeJML435GM7,Clear Scalp & Hair Therapy Total Care Nourishi...,Bring back the old formula! This new formula i...,0.048303,0
22352,AVpfOmKwLJeJML435GM7,Clear Scalp & Hair Therapy Total Care Nourishi...,The total care original formula was the best! ...,0.048303,1
22353,AVpfOmKwLJeJML435GM7,Clear Scalp & Hair Therapy Total Care Nourishi...,I'm very disappointed with the new formula. I ...,0.048303,1
22354,AVpfOmKwLJeJML435GM7,Clear Scalp & Hair Therapy Total Care Nourishi...,I'm so glad I'm not the only one completely di...,0.048303,1
22355,AVpfOmKwLJeJML435GM7,Clear Scalp & Hair Therapy Total Care Nourishi...,I loved this shampoo!!! It used to make my hai...,0.048303,1
...,...,...,...,...,...
22718,AVpfOmKwLJeJML435GM7,Clear Scalp & Hair Therapy Total Care Nourishi...,I have had an issue with psoriasis for years. ...,0.048303,1
22719,AVpfOmKwLJeJML435GM7,Clear Scalp & Hair Therapy Total Care Nourishi...,Mostly all brands (cheaper to most expensive) ...,0.048303,1
22720,AVpfOmKwLJeJML435GM7,Clear Scalp & Hair Therapy Total Care Nourishi...,I was curious when this shampoo came out so I ...,0.048303,1
22721,AVpfOmKwLJeJML435GM7,Clear Scalp & Hair Therapy Total Care Nourishi...,"I love this product, this is the ONLY one I us...",0.048303,1
